# General Images Ripper
Salut! This notebook has the power to _rip images from almost anywhere in the web!_ Its kinda like `ripme.jar` but much more customizable and doesn't require to be updated once in a while (this notebook's up to me update it, unfortunately lol). This is a **_work in progress_**, and if you want this notebook to rip a specific site that you want, then you gotta figure that out yourself, unless you want to pay me then go to [contact me](https://arifhamed.com/contact). The functions here should suffice.

Requirements: 
- **Internet** connection
- **Google Account** 
- A **browser** to run this
- (Optional) **`curl`** (If you are running on Linux then it should be no problem). One of the download methods makes use of https://file.io, a command-line file transfer site. Very similar in function like https://wetransfer.com, but instead of email its for command-line.

Usage: 
- Run the first cell (mandatory)
- Run whatever cell that is labelled by its respective markdown block above it. 
- Sit back & relax. You can do other things while this notebook will do its magic. Google Colab will send a notification after its done (if you allowed it to do so through the browser)
- Once its done, the following would occur if your `dl_mode` in the `save_content()` function is set as:
  - `curl`
    - The `curl` command will be shown as a string in the output of the cell that has `dl_mode` set as `curl` in the `save_content()` function. Just **copy & paste** the string to your **terminal/console** of choice (works on **Ubuntu 22.04**, **fastest option**). The output files are uploaded to https://file.io for temporary storage, which will last for 24 hours, and it will delete itself after 1 download. There's much more options for uploading, check it out on their website.
  - `files` 
    - The output files will be normally downloaded through the browser (slow, but reliable if https://file.io becomes unstable or if Google Drive gets hacked)
  - `drive` 
    - This notebook may require you to mount your [**Google Drive**](https://drive.google.com) to this instance so that it can move the output files to your Google Drive instead. This is good if you are not on a _machine that has much space_ and if you have a _lot of space to spare in your drive_. Recommended if you either use **ChromeOS**, if you **don't immediately have your main machine with you**, or if you're very **cloud-centric**. 

In [ ]:
!pip install img2pdf 

import time, pytz, os, sys, requests, codecs, json
from tqdm.notebook import tqdm
from google.colab import files
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import timedelta, datetime
from IPython.display import clear_output 
time_alpha = time.time()

# Essential to make sure webpages does not detect you as a bot (even though this
# is basically a bot). Does not work for those that require Javascript (WIP for 
# a workaround)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
def request_with_headers(url, sep="\n"): # Returns list
  return requests.get(url, headers=headers).content.decode().split(sep)

file_io_command = "" # For curl

def save_content(savename="output", type_mode="zip", dl_mode="files", pdf_args=""): 
  global file_io_command
  # clear_output() # Clears previous file_io_command if there was
  os.system(f"mkdir -p /content/{type_mode}")
  savename = f"/content/{type_mode}/{savename}.{type_mode}"
  if type_mode == "zip":
    os.system(f"zip -r {savename} /content/*.*")
  elif type_mode == "pdf":
    os.system(f"img2pdf /content/*.* -o {savename} --producer=\"Google Colab\"  {pdf_args}")
  
  # I really wanted to use `match` (released in Python 3.10) here 
  # but Google Colab only has Python 3.8.16 as of this typing
  if dl_mode == "files":
    files.download(savename)
  elif dl_mode == "curl":
    temp_str = os.popen(f'curl -F "file=@{savename}" https://file.io').read()
    # DEBUGGING print()
    print(repr(temp_str)) 
    obj_temp = json.loads(temp_str)
    file_io_command += f"curl \"{obj_temp['link']}\" --output \"{obj_temp['name']}\"; "
  elif dl_mode == "drive":
    os.system(f"mv {savename} /content/drive/MyDrive/{savename.split('/')[-1]}")
  else: #dl_mode == ""
    pass
  
  # removes all if it is a file in /content
  for i in [x for x in os.listdir("/content") if os.path.isfile(x)]:
    try: os.remove(i)
    except: pass

# !rm -rf /content/*
for i in ("pdf","zip"):
  os.system(f"rm -rf /content/*.* /content/{i}/")

### Only run this cell if you want to use your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p ~/.ssh

## junji-ito.com
As of 15 Jan 2022, these 4 are publicly available on Junji Ito's site

In [ ]:
# the lists are set to variables so that tqdm knows how to much there is inside 
# in each iteration. 
for comic in tqdm(("tomie", "uzumaki", "gyo", "remina")):
  list_focus_i = ["http"+x.split("\"")[0] for x in request_with_headers(f"https://junji-ito.com/{comic}/",sep="http") if f"{comic}-chapter" in x][::-1]
  # n is album index
  for n, i in tqdm(enumerate(list_focus_i), total=len(list_focus_i), leave=False): 
    list_focus_j = list(dict.fromkeys([x.split("\"")[0] for x in request_with_headers(i, sep='property="og:image" content="')][1:]))
    for j in tqdm(list_focus_j, total=len(list_focus_j), leave=False):
      os.system(f"wget {j} -O {str(n).zfill(2)}_{j.split('/')[-1]}")
  save_content(f"junji-ito_{comic}", type_mode="pdf", dl_mode="curl", pdf_args=f" --title=\"{comic.capitalize()}\" --author=\"Junji Ito\" --subject=\"Horror\"")

# note for ari: reconsider this line below, cuz other people might want to use other forms of dl-ing
file_io_command

## readuzumaki.com
this one has the lost chapter, and filesize is smaller

In [ ]:
for i in tqdm(range(20)):
  for j in [x.split("\"")[0] for x in request_with_headers(f"https://readuzumaki.com/manga/uzumaki-chapter-{i+1}/", sep="<meta property=\"twitter:image\" content=\"") if x[:4] == "http"]:
    os.system(f"wget -O {str(i).zfill(2)}_{j.split('/')[-1]} {j}")

save_content("readuzumaki", type_mode="pdf", dl_mode="curl", pdf_args=f" --title=\"Uzumaki\" --author=\"Junji Ito\"")
file_io_command

########################################################################################

# Caution: below this point are all WIPs

########################################################################################



## One Punch Man (manga, not the webcomic)
URL: https://ww1.onepunch.online/

In [ ]:
# !rm -rf /content/*
i_delimeter="https://ww1.onepunch.online/manga/one"
i_list = [i_delimeter+x.split("\"")[0] for x in request_with_headers(f"https://ww1.onepunch.online/", sep=i_delimeter)[1:-3]][::-1]
for n, i in tqdm(enumerate(i_list), total=len(i_list)):
  # assert n < 3
  j_delimeter = "https://cdn.hxmanga.com/file/"
  j_list = list(dict.fromkeys([j_delimeter+x.split("\"")[0] for x in request_with_headers(i, sep=j_delimeter)[1:-1]]))
  
  tempo = i.split('-chapter-')[-1][:-1]
  tempo = str(tempo).zfill(3 + tempo.count("-") * 2)

  for j in tqdm(j_list, total=len(j_list), leave=False):
    current_query = f"wget {j} -O /content/{tempo}_{j.split('/')[-1].zfill(6)}"
    os.system(current_query)
  
  save_content(f"one-punch-man_issue-{tempo}", type_mode="pdf", dl_mode="", pdf_args=f" --title=\"One Punch Man: Issue {str(i.split('-chapter-')[-1][:-1].split('-')[0]).zfill(3)}\" --creator=\"ONE\" --author=\"Yusuke Murata\" --subject=\"Action\"")

dl_mode = "drive"
###################### file.io maximum usage is 2GB, so must split liao
if dl_mode == "curl":
  def split_dir(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))
  current_size = 0
  target_dir_str = "/content/pdf/"
  target_dir = os.listdir(target_dir_str)
  for i in os.listdir(target_dir_str):
    current_size += os.path.getsize(target_dir_str+i)
  # for i in range(int(str(current_size / 1800000000).split(".")[0])+1):

  for n, i in enumerate(list(split_dir(target_dir, int(str(current_size / 1800000000).split(".")[0])+1))):
    # print(len(i))
    for j in i:
      os.system(f"mv {target_dir_str}{j} {j}")
    torget = f"one-punch-{str(n).zfill(2)}.zip"
    os.system(f"zip -r {torget} /content/*.pdf")
    temp_obj = json.loads(os.popen(f'curl -F "file=@{torget}" https://file.io').read())
    # print(temp_obj)
    file_io_command += f"curl \"{temp_obj['link']}\" --output \"{temp_obj['name']}\"; "
    os.system("rm -rf /content/*.*")
if dl_mode == "drive":
  torget = "/content/one-punch.zip"
  os.system(f"zip -r {torget} /content/pdf/")
  os.system(f"mv {torget} /content/drive/MyDrive/{torget.split('/')[-1]}")
# temp_obj = json.loads(os.popen(f'curl -F "file=@{torget}" https://file.io').read())
# print(temp_obj)
# file_io_command = f"curl \"{temp_obj['link']}\" --output \"{temp_obj['name']}\"; "
# file_io_command

## Reddit
Original: [source](https://github.com/Watchful1/Sketchpad/blob/master/postDownloader.py)

In [ ]:
subreddit = '' #@param {type:"string"}

In [ ]:
import requests
from datetime import datetime
import traceback
import time
import json
import sys

username = ""  # put the username you want to download in the quotes
subreddit = "thinkpad"  # put the subreddit you want to download in the quotes
thread_id = ""  # put the id of the thread you want to download in the quotes, it's the first 5 to 7 character string of letters and numbers from the url, like 107xayi
# leave either one blank to download an entire user's or subreddit's history
# or fill in both to download a specific users history from a specific subreddit

convert_to_ascii = False  # don't touch this unless you know what you're doing

filter_string = None
if username == "" and subreddit == "" and thread_id == "":
	print("Fill in username, subreddit or thread id")
	sys.exit(0)

filters = []
if username:
	filters.append(f"author={username}")
if subreddit:
	filters.append(f"subreddit={subreddit}")
if thread_id:
	filters.append(f"link_id=t3_{thread_id}")
filter_string = '&'.join(filters)

url = "https://api.pushshift.io/reddit/{}/search?limit=1000&order=desc&{}&before="

start_time = datetime.utcnow()  #datetime.strptime("10/05/2021", "%m/%d/%Y")
end_time = None  #datetime.strptime("09/25/2021", "%m/%d/%Y")#datetime.utcnow()


def downloadFromUrl(filename, object_type):
	print(f"Saving {object_type}s to {filename}")

	count = 0
	if convert_to_ascii:
		handle = open(filename, 'w', encoding='ascii')
	else:
		handle = open(filename, 'w', encoding='UTF-8')
	previous_epoch = int(start_time.timestamp())
	break_out = False
	while True:
		new_url = url.format(object_type, filter_string)+str(previous_epoch)
		json_text = requests.get(new_url, headers={'User-Agent': "Post downloader by /u/Watchful1"})
		time.sleep(1)  # pushshift has a rate limit, if we send requests too fast it will start returning error messages
		try:
			json_data = json_text.json()
		except json.decoder.JSONDecodeError:
			time.sleep(1)
			continue

		if 'data' not in json_data:
			break
		objects = json_data['data']
		if len(objects) == 0:
			break

		for object in objects:
			previous_epoch = object['created_utc'] - 1
			if end_time is not None and datetime.utcfromtimestamp(previous_epoch) < end_time:
				break_out = True
				break
			count += 1
			if object_type == 'comment':
				try:
					handle.write(str(object['score']))
					handle.write(" : ")
					handle.write(datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"))
					handle.write(" : u/")
					handle.write(object['author'])
					handle.write(" : ")
					handle.write(f"https://www.reddit.com{object['permalink']}")
					handle.write("\n")
					if convert_to_ascii:
						handle.write(object['body'].encode(encoding='ascii', errors='ignore').decode())
					else:
						handle.write(object['body'])
					handle.write("\n-------------------------------\n")
				except Exception as err:
					print(f"Couldn't print comment: https://www.reddit.com{object['permalink']}")
					print(traceback.format_exc())
			elif object_type == 'submission':
				try:
					handle.write(str(object['score']))
					handle.write(" : ")
					handle.write(datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"))
					handle.write(" : ")
					if convert_to_ascii:
						handle.write(object['title'].encode(encoding='ascii', errors='ignore').decode())
					else:
						handle.write(object['title'])
					handle.write(" : u/")
					handle.write(object['author'])
					handle.write(" : ")
					handle.write(f"https://www.reddit.com{object['permalink']}")
					handle.write("\n")
					if object['is_self']:
						if 'selftext' in object:
							if convert_to_ascii:
								handle.write(object['selftext'].encode(encoding='ascii', errors='ignore').decode())
							else:
								handle.write(object['selftext'])
					else:
						handle.write(object['url'])

					handle.write("\n-------------------------------\n")
				except Exception as err:
					print(f"Couldn't print post: {object['url']}")
					print(traceback.format_exc())

		if break_out:
			break

		print("Saved {} {}s through {}".format(count, object_type, datetime.fromtimestamp(previous_epoch).strftime("%Y-%m-%d")))

	print(f"Saved {count} {object_type}s")
	handle.close()

if not thread_id:
	downloadFromUrl("posts.txt", "submission")
# downloadFromUrl("comments.txt", "comment")

In [ ]:
# for i in [x[] for x in json.loads(request_with_headers(f"https://www.reddit.com/r/thinkpad.json")[0])['data']['children']]:
for i in ["https://i.redd.it"+x for x in request_with_headers(f"https://www.reddit.com/r/thinkpad.json?count=1000",sep="redd.it")]:
  print(i)

## Carousell search (first page)


In [ ]:
# replace space with "+"
query = "thinkpad%2025"

result = [codecs.decode(x.split("\"")[0], "unicode-escape") for x in request_with_headers(f"https://www.carousell.sg/search/{query}?addRecent=true&canChangeKeyword=true&includeSuggestions=true", 'http') if "anniversary" in x]


for i in result: 
  print(repr(i))
# save_content(f"carousell_{query}.zip")

---

# Audio section!!????
yeah. still wip.

In [ ]:
!apt update
!apt upgrade -y
!apt install yt-dlp

In [ ]:
# !rm -rf /content/*
# !apt install youtube-dl
# !wget https://github.com/arifhamed/static/raw/main/others/revive_music.txt

!youtube-dl -f "bestaudio/best" --extract-audio --audio-format flac --audio-quality 0 -a revive_music.txt
# print(os.popen('youtube-dl -f "bestaudio/best" --extract-audio --audio-format flac --audio-quality 0 -a revive_music.txt').read())
# save_content("music","zip","drive")

!mkdir -p /content/zip
!mv /content/*.* /content/zip/

def split_dir(a, n):
  k, m = divmod(len(a), n)
  return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))
current_size = 0
target_dir_str = "/content/zip/"
target_dir = os.listdir(target_dir_str)
for i in os.listdir(target_dir_str):
  current_size += os.path.getsize(target_dir_str+i)
# for i in range(int(str(current_size / 1800000000).split(".")[0])+1):

for n, i in enumerate(list(split_dir(target_dir, int(str(current_size / 1800000000).split(".")[0])+1))):
  # print(len(i))
  for j in i:
    os.system(f"mv {target_dir_str}{j} {j}")
  torget = f"music-{str(n).zfill(2)}.zip"
  os.system(f"zip -r {torget} /content/*.pdf")
  temp_obj = json.loads(os.popen(f'curl -F "file=@{torget}" https://file.io').read())
  # print(temp_obj)
  file_io_command += f"curl \"{temp_obj['link']}\" --output \"{temp_obj['name']}\"; "
  os.system("rm -rf /content/*.*")

file_io_command

In [ ]:
# !ps aux | grep youtube-dl
!youtube-dl